In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
import json

file = json.load(open("/content/sample_data/json_extraction_dataset_500.json", "r"))
print(file[1])

In [2]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

In [3]:
model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True)

==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

# Train the model
trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 3 | Total steps = 189
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Step,Training Loss
25,0.451300
50,0.149900
75,0.134000
100,0.122200
125,0.115600
150,0.112100
175,0.110400


In [6]:
# Save only the lightweight adapter and tokenizer
model.save_pretrained("phi3_lora_adapter")
tokenizer.save_pretrained("phi3_lora_adapter")

print("Adapter saved to folder: phi3_lora_adapter")

Adapter saved to folder: phi3_lora_adapter


In [12]:
!ls ./phi3_lora_adapter

adapter_config.json	   chat_template.jinja	    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  tokenizer.model


In [16]:
import os
import subprocess
import sys

# 1. Kill everything
!pkill -f vllm

# 2. Start the server forcing V0 engine and low VRAM usage
# We set VLLM_USE_V1=0 directly in the environment string
cmd = "VLLM_USE_V1=0 python -m vllm.entrypoints.openai.api_server " \
      "--model unsloth/Phi-3-mini-4k-instruct-bnb-4bit " \
      "--enable-lora " \
      "--lora-modules phi_task=./phi3_lora_adapter " \
      "--max-lora-rank 64 " \
      "--port 8000 " \
      "--trust-remote-code " \
      "--gpu-memory-utilization 0.4 " \
      "--max-model-len 256 " \
      "--enforce-eager"

with open("vllm.log", "w") as log_file:
    subprocess.Popen(cmd, shell=True, stdout=log_file, stderr=log_file)

print("🚀 Server starting with VLLM_USE_V1=0...")

🚀 Server starting with VLLM_USE_V1=0...


In [ ]:
!tail -f vllm.log

In [27]:
!netstat -tlpn | grep 8000

In [25]:
import requests
import json

# We hardcode the full address directly in the post call to ignore any other 'url' variables
try:
    response = requests.post(
        "http://127.0.0.1",
        json={
            "model": "phi_task",
            "messages": [
                {"role": "user", "content": "### Input: Extract the product information:\n<div class='product'><h2>Asus ROG Strix</h2><span class='price'>$1106</span></div>\n### Output:"}
            ],
            "temperature": 0
        },
        timeout=60
    )

    if response.status_code == 200:
        # vLLM follows OpenAI format: 'choices' is a list, content is in 'message'
        result = response.json()['choices'][0]['message']['content']
        print("✅ Success! Extracted Output:")
        print(result)
    else:
        print(f"❌ Server Error {response.status_code}: {response.text}")

except Exception as e:
    print(f"⚠️ Connection failed: {e}")
    print("If this still says 'port=80', restart your Colab runtime (Runtime > Restart session) and re-run ONLY the server and this cell.")

⚠️ Connection failed: HTTPConnectionPool(host='127.0.0.1', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7cd56e12bf20>: Failed to establish a new connection: [Errno 111] Connection refused'))
If this still says 'port=80', restart your Colab runtime (Runtime > Restart session) and re-run ONLY the server and this cell.


In [28]:
import sys, os, subprocess, time

# 1. Clean up
!pkill -f vllm

# 2. Hard-limit memory for T4
os.environ["VLLM_USE_V1"] = "0"
os.environ["NCCL_P2P_DISABLE"] = "1"

# 3. Start with MINIMAL VRAM (30%) and MINIMAL context (128 tokens)
# This ensures it fits on a T4 alongside the base model
cmd = (f"{sys.executable} -m vllm.entrypoints.openai.api_server "
      f"--model unsloth/Phi-3-mini-4k-instruct-bnb-4bit "
      f"--enable-lora --lora-modules phi_task=./phi3_lora_adapter "
      f"--max-lora-rank 64 --port 8000 --trust-remote-code "
      f"--gpu-memory-utilization 0.3 --max-model-len 128 --enforce-eager")

with open("vllm.log", "w") as log_file:
    subprocess.Popen(cmd, shell=True, stdout=log_file, stderr=log_file)

print("🚀 Server starting in Safe Mode... wait 60 seconds.")

# 4. Wait for it to actually bind the port
time.sleep(60)
!netstat -tlpn | grep 8000

🚀 Server starting in Safe Mode... wait 60 seconds.


In [49]:
!netstat -tlpn | grep 8000

tcp        0      0 0.0.0.0:8000            0.0.0.0:*               LISTEN      21281/python3       


In [50]:
import requests

url = "http://127.0.0.1:8000/v1/chat/completions"

payload = {
    "model": "phi_task",
    "messages": [
        {"role": "user", "content": "Extract: <h2>Asus</h2>"}
    ],
    "temperature": 0
}

response = requests.post(url, json=payload, timeout=30)

if response.status_code == 200:
    print("✅ Connection worked!")
    print(response.json()["choices"][0]["message"]["content"])
else:
    print(f"❌ Server returned error {response.status_code}: {response.text}")


✅ Connection worked!
 {"name": "Asus", "category": "smartphones", "price": "$1106"}
